Classifing hate speech tweets using Support Vector Machine 

In [1]:
import pandas as pd
import requests
import io
import sklearn.model_selection as ms
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
# Sources used:
# https://medium.com/@vasista/sentiment-analysis-using-svm-338d418e3ff1

In [3]:
# import pandas as pd
# import requests
# import io
    
# Downloading the csv file from GitHub
url = "https://raw.githubusercontent.com/Aaron9812/Data_mining/main/data/220502_train_data_preprocessed.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('utf-8')), sep=";")

# Printing out the first rows of the dataframe
# print (df.head())

   id  label                                              tweet  n_mentions  \
0   1      0   @user when a father is dysfunctional and is s...           1   
1   2      0  @user @user thanks for #lyft credit i can't us...           2   
2   3      0                                bihday your majesty           0   
3   4      0  #model   i love u take with u all the time in ...           0   
4   5      0             factsguide: society now    #motivation           0   

                                hashtags  \
0                                ['run']   
1  ['lyft', 'disapointed', 'getthanked']   
2                                     []   
3                              ['model']   
4                         ['motivation']   

                                 without_puctioation  \
0   user when a father is dysfunctional and is so...   
1  user user thanks for lyft credit i cant use ca...   
2                                bihday your majesty   
3  model   i love u take with u all 

In [4]:
features = [
"tweet",
"hashtags",
"without_puctioation",
"tweet_lower",
"tweet_token",
"clean_token",
"clean_hashtags",
"stemmed_tokens",
"stemmed_hashtags",
"lemmatized_tokens",
"lemmatized_hashtags"
]
X = df[features]
y = df.label
X.head()


,tweet,hashtags,without_puctioation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
0,@user when a father is dysfunctional and is s...,['run'],user when a father is dysfunctional and is so...,user when a father is dysfunctional and is so...,"['when', 'father', 'dysfunctional', 'is', 'sel...","['father', 'dysfunctional', 'selfish', 'drags'...",['run'],"['father', 'dysfunct', 'selfish', 'drag', 'kid...",['run'],"['father', 'dysfunctional', 'selfish', 'drag',...",['run']
1,@user @user thanks for #lyft credit i can't us...,"['lyft', 'disapointed', 'getthanked']",user user thanks for lyft credit i cant use ca...,user user thanks for lyft credit i cant use ca...,"['user', 'for', 'credit', 'cant', 'cause', 'do...","['user', 'credit', 'cant', 'cause', 'dont', 'w...","['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'caus', 'dont', 'wh...","['lyft', 'disapoint', 'getthank']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","['lyft', 'disapointed', 'getthanked']"
2,bihday your majesty,[],bihday your majesty,bihday your majesty,['your'],[],[],[],[],[],[]
3,#model i love u take with u all the time in ...,['model'],model i love u take with u all the time in u...,model i love u take with u all the time in u...,"['i', 'with', 'u', 'all', 'time', 'urð\x9f\x93...","['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model']
4,factsguide: society now #motivation,['motivation'],factsguide society now motivation,factsguide society now motivation,"['society', 'motivation']","['society', 'motivation']",['motivation'],"['societi', 'motiv']",['motiv'],"['society', 'motivation']",['motivation']


In [5]:
'''
Train-Test-Split (source vs. code here)

trainData --> X_train
testData --> X_test
trainData['Label'] --> y_train
testData['Label'] --> y_test
'''

"\ntrainData --> X_train\ntestData --> X_test\ntrainData['Label'] --> y_train\ntestData['Label'] --> y_test\n"

In [7]:
(X_train, X_test, y_train, y_test) = ms.train_test_split(X, y, test_size=0.2, random_state = 17, stratify=y)

In [8]:
print("# rows dataset: ", len(df))
print(len(X_train), len(X_test), len(y_train), len(y_test))
print("sum: ", (len(y_train) + len(y_test)))

# rows dataset:  31962
25569 6393 25569 6393
sum:  31962


In [9]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [10]:
for feature in features:
    vectors_train = vectorizer.fit_transform(X_train[feature])
    vectors_test = vectorizer.transform(X_test[feature])
    # Perform classification with SVM, kernel=linear
    classifier_linear = svm.SVC(kernel='linear')
    classifier_linear.fit(vectors_train, y_train)

    prediction_linear = classifier_linear.predict(vectors_test)

    # results
    report = classification_report(y_test, prediction_linear, output_dict=True)
    print(feature,": ", report['1'])


tweet :  {'precision': 0.8764940239043825, 'recall': 0.49107142857142855, 'f1-score': 0.6294706723891274, 'support': 448}
hashtags :  {'precision': 0.8838383838383839, 'recall': 0.390625, 'f1-score': 0.5417956656346749, 'support': 448}
without_puctioation :  {'precision': 0.8906882591093117, 'recall': 0.49107142857142855, 'f1-score': 0.6330935251798561, 'support': 448}
tweet_lower :  {'precision': 0.8906882591093117, 'recall': 0.49107142857142855, 'f1-score': 0.6330935251798561, 'support': 448}
tweet_token :  {'precision': 0.8536585365853658, 'recall': 0.3125, 'f1-score': 0.45751633986928114, 'support': 448}
clean_token :  {'precision': 0.8466257668711656, 'recall': 0.3080357142857143, 'f1-score': 0.4517184942716857, 'support': 448}
clean_hashtags :  {'precision': 0.883248730964467, 'recall': 0.38839285714285715, 'f1-score': 0.5395348837209302, 'support': 448}
stemmed_tokens :  {'precision': 0.863905325443787, 'recall': 0.32589285714285715, 'f1-score': 0.47325769854132904, 'support': 4